In [ ]:
# Safe shutdown + restart prompt
get_ipython().kernel.do_shutdown(restart=True)  # then re-run cells from the top


{'status': 'ok', 'restart': True}

: 

# Part 4: Production‑Ready Pipeline Development

In this notebook we demonstrate how to use the **production_pipeline** package developed in Part 4 to build, train and deploy a Telco customer churn prediction model. The goal is to show how the modular pipeline orchestrates data preprocessing, model training with cross‑validation and hyperparameter tuning, evaluation on a hold‑out test set, and real‑time inference.

We use the provided churn dataset with missing values already handled (`ChurnData_Missing_Handled.csv`). The pipeline will drop the unique `customerID` column, impute missing values, bin the `tenure` feature into categories, one‑hot encode categorical variables, scale numeric features, train a decision tree model (with optional hyperparameter search), and evaluate its performance. Finally, we save the preprocessing pipeline and model, load them back and perform batch and single‑sample predictions.

In [1]:
import pandas as pd
from production_pipeline.config import DataConfig, PreprocessingConfig, ModelConfig
from production_pipeline.training_pipeline import TrainingPipeline
from production_pipeline.streaming_inference_pipeline import StreamingInferencePipeline


In [2]:
# Define path to the cleaned dataset
data_path = 'Data/Processed/ChurnData_Missing_Handled.csv'
# Load a few rows to inspect the data
df_raw = pd.read_csv(data_path)
df_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# Set up configuration objects
data_cfg = DataConfig(file_path=data_path, test_size=0.2, val_size=0.1, random_state=42, stratify=True)
prep_cfg = PreprocessingConfig(missing_strategy='median', encoding_strategy='onehot', scaling_strategy='standard', binning_strategy='tenure')
# Use a DecisionTreeClassifier for a quick demonstration
model_cfg = ModelConfig(model_type='decision_tree', params={})
# Optionally provide a hyperparameter grid for max_depth
hyperparams = {'max_depth': [3, 5, 7, None]}
# Create and run the training pipeline
training_pipeline = TrainingPipeline(data_cfg, prep_cfg, model_cfg, hyperparams=hyperparams, threshold=0.5, save_dir='Artifacts/TrainPipeline')
results = training_pipeline.run()
results_keys = list(results.keys())
results_keys

['pipeline_path',
 'model_path',
 'cv_results',
 'validation_evaluation',
 'test_evaluation']

In [4]:
# Display evaluation results
print('Validation metrics:', results['validation_evaluation'])
print('Test metrics:', results['test_evaluation'])

# The cross‑validation results for hyperparameter tuning can be inspected as a DataFrame
import pandas as pd
cv_df = pd.DataFrame(results['cv_results'])
# Show only columns relevant for the primary metric (F1)
cols_of_interest = [col for col in cv_df.columns if 'mean_test_f1' in col or col.startswith('param')]
cv_df[cols_of_interest]


Validation metrics: EvaluationResults(accuracy=0.7749796913078798, precision=0.5757575757575758, recall=0.581039755351682, f1=0.578386605783866, roc_auc=0.8123359313685692, pr_auc=0.5881525519432036, confusion_matrix=array([[764, 140],
       [137, 190]]))
Test metrics: EvaluationResults(accuracy=0.8011363636363636, precision=0.6111111111111112, recall=0.7021276595744681, f1=0.6534653465346535, roc_auc=0.8688767936665018, pr_auc=0.6874513921859718, confusion_matrix=array([[108,  21],
       [ 14,  33]]))


,param_max_depth,params,mean_test_f1
0,3,{'max_depth': 3},0.506193
1,5,{'max_depth': 5},0.551543
2,7,{'max_depth': 7},0.529992
3,None,{'max_depth': None},0.494649


In [5]:
# Load saved pipeline and model for inference
pipeline_path = results['pipeline_path']
model_path = results['model_path']
# Initialize streaming inference pipeline
sip = StreamingInferencePipeline(pipeline_path, model_path, threshold=0.5)

# Perform batch prediction on the first 5 customers
samples = df_raw.head(5).drop(columns=['Churn'])
predictions_batch = sip.predict_batch(samples)
predictions_batch

# Perform single‑sample prediction on a dictionary
single_sample = samples.iloc[0].to_dict()
prediction_single = sip.predict(single_sample)
prediction_single


PredictionResult(probability=0.5098814229249012, prediction=1)

## Summary

This notebook illustrates how the **production_pipeline** package implements a production‑ready workflow for Telco customer churn prediction:

- **Modular Preprocessing:** The `DataPipeline` removes identifier columns, imputes missing values, bins the tenure feature, encodes categoricals, and scales numerical variables.
- **Configurable Training:** The `TrainingPipeline` orchestrates data ingestion, preprocessing, splitting, model construction, cross‑validated training, optional hyperparameter tuning, evaluation, and persistence.
- **Robust Evaluation:** We compute metrics appropriate for imbalanced classification (precision, recall, F1, ROC AUC, PR AUC) on a hold‑out test set.
- **Model Persistence & Inference:** The fitted pipeline and model are saved to disk. The `StreamingInferencePipeline` loads these artifacts and provides both batch and single‑sample predictions with configurable thresholds, returning probabilities and hard class labels.

This modular architecture allows the churn prediction system to be extended or replaced with different preprocessing strategies, models or evaluation criteria without modifying the pipeline orchestration code.
